---



# <font color='#ECA702'>**Análisis Multimodal de Síntomas en la Enfermedad de Parkinson**</font>

In [ ]:
#@title **Instalar paquetes necesarios**
%%capture 
! pip install torchmetrics
! pip install torchinfo
! pip install wandb -Uq

In [1]:
#@title **Importar librerías**

# Manipulación de datos
import os
import cv2
import numpy as np
import pandas as pd
from zipfile import ZipFile

# Pytorch essentials
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

# Pytorch essentials for datasets.
from torch.utils.data import Dataset, DataLoader

# Pytorch metrics
from torchmetrics.classification import BinaryAccuracy, BinaryF1Score, BinaryRecall, BinaryPrecision

# Utils
import matplotlib.pyplot as plt
# import copy

# sklearn
from sklearn.model_selection import GroupKFold

# wandb
import wandb

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(


In [3]:
#@title **Cargamos los datos desde Drive**

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/data_parkinson')
print(os.getcwd())

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1XRruCPRodR4OhRCRZYKsyODqa_GuBkfC/data_parkinson


In [4]:
#@title **Extraer el zip**

os.mkdir('../../../../data')
os.popen('cp data.csv ../../../../data/data.csv')
target_path = os.path.join('../../../../data') # Aquí es donde van a quedar los datos
# deben quedar fuera de drive para poder acceder a ellos 'localmente'

# Unzip the downloaded files
zip_file_path = os.path.join('AudioVisualData_v7.zip')
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_path)

# # Me muevo hasta donde quedaron los datos
%cd ../../../../data
# Verifico
%ls

/content/data
AudioVisualData_v7/  data.csv


# <font color='#4C5FDA'>**1. Procesamiento del dataset**</font>

## <font color='#52F17F'>**Creación del Dataset Pandas auxiliar**</font>

In [2]:
data = pd.read_csv("data.csv", index_col='patient')

def string2array(string: str):
  """ Convierte un string con datos a numpy array """
  string = string.strip('[]')
  return np.fromstring(string, sep=',')

data['flatten_log_mel_spectogram'] = data['flatten_log_mel_spectogram'].apply(string2array)
data.head()

,audio_path,frames_path,label,flatten_log_mel_spectogram
patient,,,,
C2,AudioVisualData_v7/Control/C2/Vowels/audio/C2-...,AudioVisualData_v7/Control/C2/Vowels/frames/C2...,0.0,"[-60.66636657714844, -19.715749740600586, -24...."
C2,AudioVisualData_v7/Control/C2/Vowels/audio/C2-...,AudioVisualData_v7/Control/C2/Vowels/frames/C2...,0.0,"[-61.8838005065918, -28.279430389404297, -28.8..."
C2,AudioVisualData_v7/Control/C2/Vowels/audio/C2-...,AudioVisualData_v7/Control/C2/Vowels/frames/C2...,0.0,"[-59.091400146484375, -25.001014709472656, -28..."
C2,AudioVisualData_v7/Control/C2/Vowels/audio/C2-...,AudioVisualData_v7/Control/C2/Vowels/frames/C2...,0.0,"[-60.556915283203125, -20.145050048828125, -20..."
C2,AudioVisualData_v7/Control/C2/Vowels/audio/C2-...,AudioVisualData_v7/Control/C2/Vowels/frames/C2...,0.0,"[-69.82708740234375, -20.313282012939453, -26...."


## <font color='#52F17F'>**Creación del Dataset PyTorch**</font>

In [3]:
"""
Constante sacada de la media de cantidad de frames de todos los vídeos multiplicada por 0.1
para tener el 10%
"""

FRAMES_PROMEDIO = 14

class PatientDataset(Dataset):
  def __init__(self, dataframe, n_frames = FRAMES_PROMEDIO):
    self.dataframe = dataframe
    # self.transform = transform
    self.n_frames = n_frames

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, index):

    """ Carga de la imagen """

    # Leemos el audio
    audio = self.dataframe['flatten_log_mel_spectogram'][index].astype(np.float32)
    audio = audio.reshape((90, 10))
    audio = torch.from_numpy(audio).unsqueeze(0)

    """ Carga de los frames """

    video_path = os.path.join(self.dataframe['frames_path'][index])

    frames_path = sorted(os.listdir(video_path))[:self.n_frames]
    frames = []
    for frame in frames_path:
      frame_path = os.path.join(self.dataframe['frames_path'][index], frame)
      # print(frame_path)
      frame = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)
      frame = cv2.resize(frame, (112, 112)) # Resize a 112x112
      frame = frame / 255. # Normalizamos entre 0 y 1
      frames.append(frame)

    frames = np.expand_dims(np.stack(frames), axis=1).astype('float32')
    frames = frames.transpose(1, 0, 2, 3)
    frames = torch.from_numpy(frames)

    """ Consiguiendo el label """

    # Para conseguir la etiqueta simplemente me tengo que fijar en la ruta. Si es 'Parkinson' 1
    # sino 0.

    label = 1 if 'Parkinson' in self.dataframe['audio_path'][index] else 0

    return audio, frames, label
    

In [4]:
complete_dataset = PatientDataset(data)
print(complete_dataset.__len__())
print(complete_dataset[0][0].shape, complete_dataset[0][1].shape , complete_dataset[0][2])

1091
torch.Size([1, 90, 10]) torch.Size([1, 14, 112, 112]) 0


## <font color='#52F17F'>**Partición de los datos**</font>

In [5]:
# Extraer los datos para GroupKFold
X = [i for i in range(len(data))]
y = data['label'].values
groups = data.index.values

groupk_folds = 7
gkf = GroupKFold(n_splits=groupk_folds)

# Realizar la validación cruzada por grupos
for i, (train_index, test_index) in enumerate(gkf.split(X, y, groups), 1):
    train_groups = groups[train_index]
    test_groups = groups[test_index]

    print(f"División {i}:")
    print("  Pacientes en entrenamiento:", np.unique(train_groups))
    print("  Pacientes en prueba:", np.unique(test_groups))
    print("  Número de pacientes en entrenamiento:", len(np.unique(train_groups)))
    print("  Número de pacientes en prueba:", len(np.unique(test_groups)))
    print()


División 1:
  Pacientes en entrenamiento: ['C0' 'C1' 'C2' 'C3' 'C4' 'C6' 'P0' 'P1' 'P4' 'P5' 'P6' 'P7']
  Pacientes en prueba: ['C5' 'P8']
  Número de pacientes en entrenamiento: 12
  Número de pacientes en prueba: 2

División 2:
  Pacientes en entrenamiento: ['C0' 'C1' 'C2' 'C3' 'C5' 'C6' 'P0' 'P1' 'P4' 'P5' 'P6' 'P8']
  Pacientes en prueba: ['C4' 'P7']
  Número de pacientes en entrenamiento: 12
  Número de pacientes en prueba: 2

División 3:
  Pacientes en entrenamiento: ['C0' 'C1' 'C2' 'C4' 'C5' 'C6' 'P0' 'P1' 'P4' 'P5' 'P7' 'P8']
  Pacientes en prueba: ['C3' 'P6']
  Número de pacientes en entrenamiento: 12
  Número de pacientes en prueba: 2

División 4:
  Pacientes en entrenamiento: ['C0' 'C1' 'C3' 'C4' 'C5' 'C6' 'P0' 'P1' 'P4' 'P6' 'P7' 'P8']
  Pacientes en prueba: ['C2' 'P5']
  Número de pacientes en entrenamiento: 12
  Número de pacientes en prueba: 2

División 5:
  Pacientes en entrenamiento: ['C0' 'C2' 'C3' 'C4' 'C5' 'C6' 'P0' 'P1' 'P5' 'P6' 'P7' 'P8']
  Pacientes en prueba: [

In [6]:
def make_folds(data:pd.DataFrame):
    # Extraer los datos para GroupKFold
    X = np.array([i for i in range(len(data))])
    y = data['label'].values
    groups = data.index.values

    folds_dict = {}
    groupk_folds = 7
    gkf = GroupKFold(n_splits=groupk_folds)

    # Realizar la validación cruzada por grupos
    for i, (train_index, test_index) in enumerate(gkf.split(X, y, groups), 1):        
        fold_name = f"fold_{i}"
        folds_dict[fold_name] = {
            'train': train_index,
            'test': test_index
        }

    return folds_dict

def make_subdataframes(data:pd.DataFrame, folds:dict):
  # Crear subdataframes
  subdataframes = {}

  for fold_name, indices in folds.items():
      train_df = data.iloc[indices['train']]
    #   val_df = data.iloc[indices['val']]
      test_df = data.iloc[indices['test']]
      
      subdataframes[fold_name] = {
          'train': train_df,
        #   'val': val_df,
          'test': test_df
      }
  
  return subdataframes

In [7]:
def get_data(slice=1, fold:int=None):

    # Generate folds
    folds = make_folds(data)

    # Create subdataframes  
    subdataframes = make_subdataframes(data, folds)

    if not fold:
      fold = np.random.choice(range(1, 8))

    fold_name = f'fold_{fold}'
    print(f"FOLD {fold}\n-------------------------------")

    train_dataset = PatientDataset(subdataframes[fold_name]['train'])
    test_dataset = PatientDataset(subdataframes[fold_name]['test'])
    
    # test with less data, it helped me to set up the experiments faster if slice=1
    # then it returns the complete dataset
    train_dataset = torch.utils.data.Subset(train_dataset, 
                                            indices=range(0, len(train_dataset), slice))
    test_dataset = torch.utils.data.Subset(test_dataset, 
                                            indices=range(0, len(test_dataset), slice))

    return train_dataset, test_dataset

def make_loader(dataset, batch_size):
    loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True,
                        pin_memory=True, num_workers=2)
    return loader

# <font color='#ECA702'>**2. Deep Learning 🤖**</font>

## <font color='#EB9A54'>**Multimodal fully connected model**</font>

In [27]:
class MiddleModule(nn.Module):
  def __init__(self, in_features, out_features):
    super().__init__()
    self.fc = nn.Linear(in_features, out_features)
    self.relu = nn.ReLU(inplace=True)


  def forward(self, x):
    x = self.fc(x)
    x = self.relu(x)
    return x

class MultimodalModel(nn.Module):
  def __init__(self, n_classes):
    super().__init__()
    self.fc1 = nn.Linear((1 * 14 * 112 * 112) + (1 * 90 * 10), 1024)
    self.features_module = nn.Sequential(
      MiddleModule(1024, 512),
      MiddleModule(512, 256),
      MiddleModule(256, 128),
    )
    self.fc4 = nn.Linear(128, n_classes)


  def forward(self, audio, video):
    # https://stackoverflow.com/questions/42479902/what-does-view-do-in-pytorch
    audio = audio.view(audio.size(0), -1) # Aplane los audios conservando el batch
    # devuelve: # batch_size x (90*10)

    video = video.view(video.size(0), -1) # Aplane los videos conservando el batch
    # devuelve: # batch_size x (14*112*112)

    combined = torch.cat((video, audio), dim=1) # https://pytorch.org/docs/stable/generated/torch.cat.html
    x = F.relu(self.fc1(combined))
    x = self.features_module(x)
    x = self.fc4(x)
    return x

In [28]:
# Test al modelo, para ver si nos entrega la salida esperada

input_audio = torch.rand([2, 1, 90, 10])
input_video = torch.rand([2, 1, 14, 112, 112])
print(f"Entrada audio: {input_audio.size(), input_audio.dtype}")
print(f"Entrada video: {input_video.size(), input_video.dtype}")

model = MultimodalModel(n_classes=1)
ouput = model(input_audio, input_video)
print(f"Salida: {ouput.size()}")

Entrada audio: (torch.Size([2, 1, 90, 10]), torch.float32)
Entrada video: (torch.Size([2, 1, 14, 112, 112]), torch.float32)
Salida: torch.Size([2, 1])


In [29]:
print(model)

MultimodalModel(
  (fc1): Linear(in_features=176516, out_features=196, bias=True)
  (features_module): Sequential(
    (0): MiddleModule(
      (fc): Linear(in_features=196, out_features=512, bias=True)
      (relu): ReLU(inplace=True)
    )
    (1): MiddleModule(
      (fc): Linear(in_features=512, out_features=256, bias=True)
      (relu): ReLU(inplace=True)
    )
  )
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)


In [30]:
summary(model, (input_video.size(), input_audio.size()))

Layer (type:depth-idx)                   Output Shape              Param #
MultimodalModel                          [2, 1]                    --
├─Linear: 1-1                            [2, 196]                  34,597,332
├─Sequential: 1-2                        [2, 256]                  --
│    └─MiddleModule: 2-1                 [2, 512]                  --
│    │    └─Linear: 3-1                  [2, 512]                  100,864
│    │    └─ReLU: 3-2                    [2, 512]                  --
│    └─MiddleModule: 2-2                 [2, 256]                  --
│    │    └─Linear: 3-3                  [2, 256]                  131,328
│    │    └─ReLU: 3-4                    [2, 256]                  --
├─Linear: 1-3                            [2, 1]                    257
Total params: 34,829,781
Trainable params: 34,829,781
Non-trainable params: 0
Total mult-adds (M): 69.66
Input size (MB): 1.41
Forward/backward pass size (MB): 0.02
Params size (MB): 139.32
Estimated Total 

## <font color='#EB9A54'>**Utils functions and hyperparameters**</font>

In [31]:
# Hiperparametros
LEARNING_RATE = 0.0001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 48

In [ ]:
def train_log(loss, accuracy, step, current):
    """ Log the metrics for the current batch into wandb

    Args:
        loss: the value of the loss at current batch
        accuracy: the value of the accuracy at current batch
        step: actual step
        current: actual batch
    """

    # Where the magic happens
    wandb.log({"step":step, "train_loss": loss, "train_accuracy": accuracy})
    print(f"train loss: {loss:.3f} accuracy: {accuracy:.3f} [after {current} batches]")

In [32]:
#@title **train batch**

def train_batch(audios, videos, labels, model, optimizer, criterion, metrics_fn):
  audios, videos, labels = audios.to(DEVICE), videos.to(DEVICE), labels.to(DEVICE)

  # Forward pass ➡
  outputs = model(audios, videos)
  loss = criterion(outputs, labels.unsqueeze(1).float())
  accuracy = metrics_fn(outputs, labels.unsqueeze(1).float())

  # Backward pass ⬅
  optimizer.zero_grad()
  loss.backward()

  # Step with optimizer
  optimizer.step()

  return loss, accuracy

In [33]:
def train(model, train_loader, criterion, optimizer, accuracy_fn, epochs, losses=None):
    # Initialize the step counter 
    step = 0
    # 4 means that I am going to make 4 logs of the metrics when training
    n_prints = int(len(train_loader)/4)

    for t in range(epochs):
      print(f"Epoch {t+1}\n-------------------------------")
      
      train_loss, train_accuracy = 0, 0
      model.train()
      for batch, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        audios, videos, labels = data
        loss, accuracy = train_batch(audios, videos, labels, model, optimizer, criterion, accuracy_fn)
        train_loss += loss.item()
        train_accuracy += accuracy

        # Report metrics every n_prints batch 
        if batch % n_prints == n_prints-1:
            train_log(train_loss/(batch+1), train_accuracy/(batch+1), step, batch)
            if losses:
              losses.append(train_loss/(batch+1))
            # print(f"train loss: {loss:.3f} accuracy: {accuracy:.3f} [after {batch+1} batches]")            
            # Increment the step after logging
            step += 1

In [34]:
#@title **test loop**

def test(model, test_loader, accuracy_fn, f1_score_fn, recall_fn, precision_fn):
    model.eval()

    # Run the model on some test examples
    num_batches = len(test_loader)
    test_accuracy, test_f1, test_recall, test_precision = 0, 0, 0, 0

    # Disable gradient calculation
    with torch.no_grad():
        for audios, videos, labels in test_loader:
            audios, videos, labels = audios.to(DEVICE), videos.to(DEVICE), labels.to(DEVICE)
            outputs = model(audios, videos)
            test_accuracy += accuracy_fn(outputs, labels.unsqueeze(1).float())
            test_f1 += f1_score_fn(outputs, labels.unsqueeze(1).float())
            test_recall += recall_fn(outputs, labels.unsqueeze(1).float())
            test_precision += precision_fn(outputs, labels.unsqueeze(1).float())

        # Average the metrics over all batches
        test_accuracy /= num_batches
        test_f1 /= num_batches
        test_recall /= num_batches
        test_precision /= num_batches

        wandb.log({"test_accuracy": test_accuracy, "test_f1": test_f1,
            "test_recall": test_recall, "test_precision": test_precision})
        print(f"test accuracy: {test_accuracy:.3f} recall: {test_recall:.3f} precision: {test_precision:.3f} f1: {test_f1:.3f} [after {num_batches} batches]")

    return test_accuracy, test_f1, test_recall, test_precision

In [35]:
def make(config, fold=None):

  # Make the data
  train, test = get_data(slice=1, fold=fold)
  train_loader = make_loader(train, batch_size=config.batch_size)
  test_loader = make_loader(test, batch_size=config.batch_size)

  # Creacion del modelo y lo movemos a la GPU o CPU
  model = MultimodalModel(n_classes=1).to(DEVICE)
  
  # Funcion de perdida (loss)
  criterion = nn.BCEWithLogitsLoss() # https://shivambaldha.medium.com/binary-classification-with-pytorch-85089b284940

  # Algoritmos de optimización del gradiente
  # optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
  optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

  # Make metrics
  accuracy_fn = BinaryAccuracy().to(DEVICE)
  f1_score_fn = BinaryF1Score().to(DEVICE)
  recall_fn = BinaryRecall().to(DEVICE)
  precision_fn = BinaryPrecision().to(DEVICE)

  EPOCHS = config.epochs

  return model, train_loader, test_loader, criterion, optimizer, accuracy_fn, f1_score_fn, recall_fn, precision_fn, EPOCHS

In [ ]:
def model_pipeline(num, sweep_id, sweep_run_name, hyperparameters):

    # tell wandb to get started
    run_name = f'{sweep_run_name}--{num}'
    with wandb.init(config=hyperparameters, group=sweep_id, job_type=sweep_run_name, name=run_name, reinit=True):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, test_loader, criterion, optimizer, accuracy_fn, f1_score_fn, recall_fn, precision_fn, epochs = make(config, num)
        # print(model)

        # and use them to train the model
        train(model, train_loader, criterion, optimizer, accuracy_fn, epochs)
            
        # get metrics of the model    
        test_accuracy, test_f1, test_recall, test_precision = test(model, test_loader, accuracy_fn, f1_score_fn, recall_fn, precision_fn)

    return test_accuracy, test_f1, test_recall, test_precision

In [ ]:
def reset_wandb_env():
    exclude = {
        "WANDB_PROJECT",
        "WANDB_ENTITY",
        "WANDB_API_KEY",
    }
    for key in os.environ.keys():
        if key.startswith("WANDB_") and key not in exclude:
            del os.environ[key]

def cross_validate(config):

    sweep_run = wandb.init(config=config, project="dip-project") # Inicio el sweep
    sweep_id = sweep_run.sweep_id or "unknown" # Agarro el id del sweep
    sweep_url = sweep_run.get_sweep_url() # Agarro el url del sweep
    project_url = sweep_run.get_project_url() # Agarro la url del proyecto del sweep
    sweep_group_url = f'{project_url}/groups/{sweep_id}' # Armo un string con la url del
    # proyecto y el id del sweep, para poder agrupar supongo 
    sweep_run.notes = sweep_group_url # Asigno en las notas del sweep la ruta que acabe
    # de crear
    sweep_run.save() # Guardo el sweep con las rutas y esas cosas hechas anteriormente
    sweep_run_name = sweep_run.name or sweep_run.id or "unknown_2" # Armo el string con el
    # nombre del run del sweep
    sweep_run_id = sweep_run.id # Consigo el id del run del sweep
    sweep_run.finish() # Puaso el sweep
    wandb.sdk.wandb_setup._setup(_reset=True) #  resets the settings which are set during the
    # sweep run initialization. This is crucial and acts as a workaround because it resets the
    # settings which causes the new run in a sweep to use the same run ID and settings upon it's initialization.

    # Diccionario para guardar las métricas de cada run
    metrics = {
        "test_accuracy": [],
        "test_recall": [],
        "test_precision": [],
        "test_f1_score": []
    }

    for fold in range(1, 8):

        reset_wandb_env() # Reinicio las variables de entorno en cada run

        # Entreno y valido cada run pasandole el sweep y la config
        test_accuracy, test_f1, test_recall, test_precision = model_pipeline(
            sweep_id=sweep_id, num=fold,
            sweep_run_name=sweep_run_name,
            hyperparameters=dict(sweep_run.config)
        )

        # Agrego las métricas del run actual
        metrics["test_accuracy"].append(test_accuracy.cpu())
        metrics["test_recall"].append(test_f1.cpu())
        metrics["test_precision"].append(test_recall.cpu())
        metrics["test_f1_score"].append(test_precision.cpu())

    # resume the sweep run
    sweep_run = wandb.init(id=sweep_run_id, resume="must")

    # Log metrics to sweep run
    for metric, values in metrics.items():
        avg_value = np.mean(values)
        std_value = np.std(values)
        # Average of each metric over all the folds in an experiment
        sweep_run.log({f"{metric}_mean": avg_value, f"{metric}_std": std_value}) 
        print(f'{metric.capitalize()}:')
        print(f'  Average: {avg_value:.2f}% (+/- {std_value:.2f}%)')
        for fold, value in enumerate(values):
            sweep_run.log({
                f'fold_{metric}': value,
                'fold': fold+1
            })

    sweep_run.finish()

In [ ]:
sweep_config = {
    'method': 'bayes',
    'name': 'sweep-gkfold',
    'metric': {
        'goal': 'maximize',
        'name': 'test_accuracy_mean'
    },
    'parameters': {
        'epochs': {'values': [5, 10, 15, 20]},
        'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 8,
                               'q': 8},
        'learning_rate': {'distribution': 'uniform',
                            'max': 0.1,
                            'min': 0.00001},
        'dataset': {'value': 'Parkison'}
    }
}

In [ ]:
wandb.login()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='ai1-project')
wandb.agent(sweep_id, function=cross_validate)

wandb.finish()